<a href="https://colab.research.google.com/github/MariaZharova/test_rec_systems/blob/main/Evaluate_SASRec_BIG_dataset_noEmb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install recommenders scrapbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 5.2 MB/s 
     |████████████████████████████████| 310 kB 54.8 MB/s 
     |████████████████████████████████| 197 kB 63.5 MB/s 
     |████████████████████████████████| 11.8 MB 68.5 MB/s 
     |████████████████████████████████| 355 kB 50.3 MB/s 
     |████████████████████████████████| 61 kB 9.8 MB/s 
     |████████████████████████████████| 4.7 MB 54.7 MB/s 
     |████████████████████████████████| 636 kB 65.4 MB/s 
     |████████████████████████████████| 12.4 MB 55.9 MB/s 
     |████████████████████████████████| 389 kB 71.3 MB/s 
     |████████████████████████████████| 6.6 MB 54.8 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 132 kB 75.5 MB/s 
     |████████████████████████████████| 106 kB 72.3 MB/s 
     |████████████████████████████████| 423 kB 72.8 MB/s 
  Created wheel for lightfm: fil

In [2]:
import warnings
warnings.filterwarnings("ignore")

import re
import sys
import os
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import pandas as pd 

from collections import defaultdict
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
from scipy.sparse import csr_matrix, load_npz

from recommenders.utils.timer import Timer
from recommenders.datasets.split_utils import filter_k_core

# Transformer Based Models
from recommenders.models.sasrec.model import SASREC

# Sampler for sequential prediction
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet

In [3]:
import random
from tqdm import tqdm

from recommenders.models.sasrec.model import Encoder
from recommenders.models.sasrec.model import LayerNormalization

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create special data format for SAS
dataS = SASRecDataSet(filename='./drive/MyDrive/Data_ML/BIG/out.txt', col_sep='\t')
# split into train, test and validation
dataS.split()

In [ ]:
# model variables
num_epochs = 40
batch_size = 200   #                                                         !!!
RANDOM_SEED = 100  # Set None for non-deterministic result
lr = 0.0001            # learning rate                                       !!!
maxlen = 50            # maximum sequence length for each user
num_blocks = 2         # number of transformer blocks
hidden_units = 150     # number of units in the attention calculation.       !!!
num_heads = 1          # number of attention heads
dropout_rate = 0.1     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient                       !!!
num_neg_test = dataS.itemnum  # number of negative examples per positive example


In [ ]:
# sample negative examples
sampler = WarpSampler(dataS.user_train, dataS.usernum, dataS.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)

In [ ]:
model = SASREC(item_num=dataS.itemnum,
               seq_max_len=maxlen,
               num_blocks=num_blocks,
               embedding_dim=hidden_units,
               attention_dim=hidden_units,
               attention_num_heads=num_heads,
               dropout_rate=dropout_rate,
               conv_dims = [hidden_units, hidden_units],         ### !!!
               l2_reg=l2_emb,
               num_neg_test=num_neg_test)

In [ ]:
with Timer() as train_time:
    t_test = model.train(dataS, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=5)